In [38]:
import numpy as np
import pandas as pd
import pickle
import tensorflow as tf
from tensorflow.keras.models import load_model

In [39]:
## Load Model 
model = load_model('model.h5')

In [40]:
# Load Encoders and Scaler

with open('label_encoder_gender.pkl','rb') as file:
    label_encoder_gender = pickle.load(file)



In [41]:
with open('onehotencoder_geo.pkl','rb') as file:
    onehotencoder_geo = pickle.load(file)

In [42]:
with open('scaler.pkl','rb') as file:
    scaler = pickle.load(file)

In [43]:
onehotencoder_geo.get_feature_names_out()

array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [55]:
# Example input data
input_data = {
    'CreditScore': 600,
    'Geography': 'France',
    'Gender': 'Male',
    'Age': 40,
    'Tenure': 3,
    'Balance': 60000,
    'NumOfProducts': 2,
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'EstimatedSalary': 50000
}

In [56]:
data = pd.DataFrame([input_data])
data

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,Male,40,3,60000,2,1,1,50000


In [57]:
data['Gender'] = label_encoder_gender.transform(data['Gender'])

In [58]:
data

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,1,40,3,60000,2,1,1,50000


In [59]:
geo_enco = onehotencoder_geo.transform(data[['Geography']]).toarray()

In [60]:
geo_enco

array([[1., 0., 0.]])

In [61]:
onehotencoder_geo.get_feature_names_out(['Geography'])

array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [62]:
geo_enco_df = pd.DataFrame(geo_enco,columns=onehotencoder_geo.get_feature_names_out(['Geography']))

In [63]:
geo_enco_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [64]:
data = pd.concat([data.drop('Geography', axis=1), geo_enco_df], axis=1)


In [65]:
data


,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [66]:
data_scaled = scaler.transform(data)

In [67]:
prediction = model.predict(data_scaled)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 92ms/step


In [68]:
prediction

array([[0.03529727]], dtype=float32)

In [72]:
prediction[0][0]

0.035297275

In [74]:
if prediction[0][0] > 0.5:
    print('The customer is likely to churn.')
else:
    print('The customer is not likely to churn.')


The customer is not likely to churn.
